# TensorFlow入門2 ロジスティック回帰実装

In [5]:
import numpy as np
import tensorflow as tf

入力1	入力2	出力

0	0	0

1	0	0

0	1	0

1	1	1

In [6]:
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [7]:
# 第一引数 → tf.float32で行列要素の数値のデータ型を指定
# 第二引数 → [None,2]で行列の形を指定
#             (Noneの部分はデータ数)(2はデータの次元)
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [10]:
# Valiableで重みとバイアスを用意（更新用）
W = tf.Variable(tf.zeros([2,1]))#初期値として0を入れている
b = tf.Variable(tf.zeros([1]))


$$h_θ(x) = g(θ^T x).\\ g(z) = \frac{1}{1+e^{−z}}.\\ J(\theta)=  \frac{1}{m}  \sum_{i=1}^{m}[−y^{(i)} log(h_θ(x^{(i)})) − (1−y^{(i)}) log(1−h_θ(x^{(i)}))]$$

In [13]:
# 
# ロジスティック回帰をTensorFlowで実装
# 

# tf.matmul()はNumpyのnp.dot()や@と同じ
y = tf.sigmoid(tf.matmul(x, W) + b)#xとwの内積+bをシグモイドまで通す
# 上記図の3段目の式
cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))
# 回帰問題で二乗和誤差関数を使用場合  tf.reduce_sum(tf.square(y - t))

#勾配降下法を用いてパラメータを最適化
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

# 学習後の結果の正解が正しいかどうかの判定と正解率の計算もデータフローグラフとして定義可
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# --------------------------------------------------------------------------
# 1行目で結果が正解かどうか判定しています。一つ一つ見ていきましょう。まずtf.equal()は引数に指定された2つの値が等しいかどうかを判定してくれます。戻り値はBool値です。tf.sign()は引数の値が正なら1、0なら0、負なら-1を返します。yが0.5以上かどうかで結果が決まるので、y-0.5とt-0.5の符号を比較しています。
# --------------------------------------------------------------------------
# print(correct_prediction) 
  # #Tensor("Equal_2:0", shape=(?, 1), dtype=bool)
# print(accuracy) 
  # #Tensor("Mean_2:0", shape=(), dtype=float32)

Tensor("Equal_2:0", shape=(?, 1), dtype=bool)
Tensor("Mean_2:0", shape=(), dtype=float32)


In [15]:
# 計算を実装していく

sess = tf.Session()#インスタンス化
sess.run(tf.global_variables_initializer())#初期化

# 学習(1000回)
for epoch in range(1000):
    sess.run(train_step, feed_dict={#sess.run(train_step) 勾配降下法
        x:x_train,
        t:y_train
    })
# 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(#accuracy numpyで返ってくる
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))
               #epoch=回数
               #acc_val=正解率(回数ごと）)

epoch: 0, Accuracy: 0.750000
epoch: 100, Accuracy: 1.000000
epoch: 200, Accuracy: 1.000000
epoch: 300, Accuracy: 1.000000
epoch: 400, Accuracy: 1.000000
epoch: 500, Accuracy: 1.000000
epoch: 600, Accuracy: 1.000000
epoch: 700, Accuracy: 1.000000
epoch: 800, Accuracy: 1.000000
epoch: 900, Accuracy: 1.000000


In [17]:
# 結果の確認
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]

print(prob)
# [[  1.96514215e-04] 1の可能性が50％以下
# [  4.90498319e-02]          〃
# [  4.90498319e-02]          〃
# [  9.31203783e-01]] 1の可能性が93%

#シグモイド関数を通しているので確率で表示されている

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9651421e-04]
 [4.9049832e-02]
 [4.9049832e-02]
 [9.3120378e-01]]


In [18]:
# 重みとバイアスの学習結果
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


In [19]:
# 途中の値が見たい場合（デバックのため）
mat = tf.matmul(x, W)
y = tf.sigmoid(mat + b)
print(sess.run(mat, feed_dict={
    x:x_train,
    t:y_train
}))

[[ 0.       ]
 [ 5.5699544]
 [ 5.5699544]
 [11.139909 ]]


In [21]:
# セッションを終了させる

sess.close()

# with tf.Session() as sess:
#     sess.run() # ここに計算の実行コードを入れていく

### NumPyによるスクラッチ実装との比較

TensorFlowを利用した場合、NumPyでのスクラッチのように更新の式ということを考える必要はなくなりました。

データフローグラフの構築は、スクラッチにおけるフォワードプロパゲーションの実装に近いと言えます。

また、シグモイド関数などよく使われるものは関数化されているため、それらを組み合わせていくだけで実装することが可能です。